In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import plotly.graph_objects as go

import cv2
import os

## Load Dataset and Setup Pipeline

In [2]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            images.append(img)
    return images

In [3]:
# Load all images
images = np.array(load_images_from_folder('dinos'))

# First two images
img1 = images[0]
img2 = images[1]

## Part 1: Two Images SfM

We're following the "guide" provided by this [GitHub's README by rohana96](https://github.com/rohana96/SfM), following an incremental SfM. However, we tried to make the different steps by ourselves.

### Previous feature extraction and matching

In [3]:
def feature_extraction_set(images):
    sift = cv2.SIFT_create()

    kp, des = [], []
    for im in images:
        kp_tmp, des_tmp = sift.detectAndCompute(im, None) # This assumes the extraction method to be from the CV2 library
        kp.append(kp_tmp)
        des.append(des_tmp)
    return kp, des # Can't turn them into a np array since their shape can be inhomogeneous

In [5]:
# Feature extraction
kp, des = feature_extraction_set(images)

In [4]:
def feature_matching_set(kp, des):
    # Initialize FLANN matching
    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params, search_params)

    matches = {} # Dict for easier access to each match
    for i in range(len(kp)):
        for j in range(i+1, len(kp)): # Only match each image with the rest, we don't need the full matrix
            matches_tmp = flann.knnMatch(des[i], des[j], k=2)

            # Lowe's ratio test
            good_matches = [m for m, n in matches_tmp if m.distance < 0.7 * n.distance]

            if len(good_matches) >= 4:
                # RANSAC to find homography and get inlier's mask
                pts1 = np.float32([kp[i][m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
                pts2 = np.float32([kp[j][m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
                _, mask = cv2.findHomography(pts1, pts2, cv2.RANSAC, 5.0)

                inliers = [good_matches[k] for k in range(len(good_matches)) if mask[k]==1]
                matches[(i,j)] = inliers
            else:
                matches[(i,j)] = []
    
    return matches

In [7]:
# Feature matching
matches = feature_matching_set(kp, des)

### Fundamental, Essential and Extrinsic decomposition

For this process, we'll assume we know the intrinsic camera parameters in one way or the other. In this exact case, we extracted it from the dataset, which isn't custom.  
Our goal is to get the extrinsic parameters. We can get them decomposing the Essential matrix into 4 possible outcomes.  
And to get the Essential matrix we apply the next formula $E=K_1^T*F*K_2$. So, to begin with, we apply the [Eight Point algorithm](https://cseweb.ucsd.edu/classes/fa22/cse252A-a/lec10.pdf) to find the Fundamental matrix.  
We don't directly calculate the Essential matrix because the images aren't calibrated.

In [8]:
# K = np.loadtxt('intrinsic_matrix.txt', dtype=float)

height, width = images.shape[1:3]
K = np.array([  # for dino
    [2360, 0, width / 2],
    [0, 2360, height / 2],
    [0, 0, 1]])

In [5]:
def normalize(pts):
    x_mean = np.mean(pts[:, 0])
    y_mean = np.mean(pts[:, 1])
    sigma = np.mean(np.sqrt((pts[:, 0] - x_mean) ** 2 + (pts[:, 1] - y_mean) ** 2))
    M = np.sqrt(2) / sigma
    T = np.array([
        [M, 0, -M * x_mean],
        [0, M, -M * y_mean],
        [0, 0, 1]
    ])
    return T

def eight_point_algorithm(pts1, pts2):

    pts1_homo = np.vstack((pts1, np.ones(pts1.shape[1]))).T
    pts2_homo = np.vstack((pts2, np.ones(pts2.shape[1]))).T

    # Normalization
    T = normalize(pts1_homo)
    T_prime = normalize(pts2_homo)


    pts1_homo = (T @ pts1_homo.T).T
    pts2_homo = (T_prime @ pts2_homo.T).T

    # x2.T*F*x1=0
    # A*f=0, f is F flattened into a 1D array
    

    # Create A
    A = np.zeros((pts1.shape[1], 9))
    for i in range(pts1.shape[1]):
        A[i] = np.array([
            pts1_homo[i,0]*pts2_homo[i,0], pts1_homo[i,1]*pts2_homo[i,0], pts1_homo[i,2]*pts2_homo[i,0],
            pts1_homo[i,0]*pts2_homo[i,1], pts1_homo[i,1]*pts2_homo[i,1], pts1_homo[i,2]*pts2_homo[i,1],
            pts1_homo[i,0]*pts2_homo[i,2], pts1_homo[i,1]*pts2_homo[i,2], pts1_homo[i,2]*pts2_homo[i,2]
            ])
    
    # Solve Af=0 using svd
    U,S,Vt = np.linalg.svd(A)
    F = Vt[-1,:].reshape((3,3))

    # Enforce rank2 constraint
    U,S,Vt = np.linalg.svd(F)
    S[-1] = 0
    F = U @ np.diag(S) @ Vt

    F = T_prime.T @ F @ T
    return F

In [10]:
# Fundamental matrix
pts1 = np.transpose([kp[0][m.queryIdx].pt for m in matches[(0,1)]])
pts2 = np.transpose([kp[1][m.trainIdx].pt for m in matches[(0,1)]])

F = eight_point_algorithm(pts1, pts2)

In [6]:
def essential_from_fundamental(K1, F, K2):
    return K1.T @ F @ K2

In [12]:
# Essential matrix
E = essential_from_fundamental(K, F, K) # In this case, the same intrinsic values apply to all images

In [7]:
def pose_from_essential(E):
    U,_,Vt = np.linalg.svd(E)
    W = np.array([[0, -1, 0], [1, 0, 0], [0, 0, 1]])
    
    Rs = [U @ W @ Vt, U @ W.T @ Vt]
    for i in range(len(Rs)):
        if np.linalg.det(Rs[i]) < 0:
            Rs[i] = Rs[i] * -1

    # Array with all possible camera poses (extrinsics)
    RTs = np.array([
        np.hstack((Rs[0], U[:, 2, np.newaxis])),
        np.hstack((Rs[0], -U[:, 2, np.newaxis])),
        np.hstack((Rs[1], U[:, 2, np.newaxis])),
        np.hstack((Rs[1], -U[:, 2, np.newaxis])),
    ])

    return RTs

In [14]:
# Get camera extrinsics from Essential matrix
RT2s = pose_from_essential(E)

In [15]:
# Define RT for camera 1 (center at world origin and matching orientation)
RT1 = np.hstack((np.eye(3), np.zeros((3, 1))))

### Disambiguation

The next step is to determine which is the right camera pose for image 2. For this, one method is to apply the cheirality condition.  
This condition states that the sign of the Z element in the camera coordinate system indicates the location of the 3D point respect to the camera, so ideally the right configuration would be that which points are all Z positive. However, due to possible correspondece noise, the best camera configuration maximizes the number of points satisfying the cheirality condition. [Reference](https://inst.eecs.berkeley.edu/~ee290t/fa19/lectures/lecture10-3-decomposing-F-matrix-into-Rotation-and-Translation.pdf)  
[Triangulation pdf](https://www.cs.cmu.edu/~16385/s17/Slides/11.4_Triangulation.pdf)

In [9]:
def linear_triangulation(K1, RT1, K2, RT2, pts1, pts2):
    # First, set all points to homogeneous
    pts1_homo = np.vstack((pts1, np.ones(pts1.shape[1])))
    pts2_homo = np.vstack((pts2, np.ones(pts2.shape[1])))

    # Calculate every projection matrix
    P1 = K1 @ RT1
    P2 = K2 @ RT2

    # Solve using svd
    pts3d = np.zeros((3, pts1.shape[1]))
    for i in range(pts1.shape[1]):
        A = np.array([pts1_homo[1,i]*P1[2,:] - P1[1,:],
            P1[0,:] - pts1_homo[0,i]*P1[2,:],
            pts2_homo[1,i]*P2[2,:] - P2[1,:],
            P2[0,:] - pts2_homo[0,i]*P2[2,:]])
        ATA = A.T @ A
        _, _, Vt = np.linalg.svd(ATA)
        pts3d[:, i] = Vt[-1, :3]/Vt[-1, -1]
    
    return pts3d

def linear_triangulation2(P1, P2, pts1, pts2):
    # First, set all points to homogeneous
    pts1_homo = np.vstack((pts1, np.ones(pts1.shape[1])))
    pts2_homo = np.vstack((pts2, np.ones(pts2.shape[1])))

    # Solve using svd
    pts3d = np.zeros((3, pts1.shape[1]))
    for i in range(pts1.shape[1]):
        A = np.array([pts1_homo[1,i]*P1[2,:] - P1[1,:],
                      P1[0,:] - pts1_homo[0,i]*P1[2,:],
                      pts2_homo[1,i]*P2[2,:] - P2[1,:],
                      P2[0,:] - pts2_homo[0,i]*P2[2,:]])
        ATA = A.T @ A
        _, _, Vt = np.linalg.svd(ATA)
        pts3d[:, i] = Vt[-1, :3]/Vt[-1, -1]
    
    return pts3d

In [17]:
K1, K2 = K, K
RT2 = RT2s[0]

pts3d = np.array([linear_triangulation(K, RT1, K, RT2, pts1, pts2) for RT2 in RT2s])

In [10]:
def reprojection(P1, P2, pts3d):
    pts3d_homo = np.vstack((pts3d, np.ones(pts3d.shape[1])))
    pts2d1_homo = np.dot(P1, pts3d_homo)
    pts2d2_homo = np.dot(P2, pts3d_homo)
    return pts2d1_homo/pts2d1_homo[-1], pts2d2_homo/pts2d2_homo[-1]

def double_disambiguation(K1, RT1, K2, RT2s, pts1, pts2, pts3d):
    max_positive_z = 0
    min_error = np.finfo('float').max
    best_RT = None
    best_pts3d = None
    P1 = K1 @ RT1

    pts1_homo = np.vstack((pts1, np.ones(pts1.shape[1])))
    pts2_homo = np.vstack((pts2, np.ones(pts2.shape[1])))

    for i in range(RT2s.shape[0]):
        P2 = K2 @ RT2s[i]
        num_positive_z = np.sum(pts3d[i][2, :] > 0)
        re1_pts2, re2_pts2 = reprojection(P1, P2, pts3d[i])

        err1 = np.sum(np.square(re1_pts2 - pts1_homo))
        err2 = np.sum(np.square(re2_pts2 - pts2_homo))

        err = err1 + err2

        if num_positive_z >= max_positive_z and err < min_error:
            max_positive_z = num_positive_z
            min_error = err
            best_RT = RT2s[i]
            best_pts3d = pts3d[i]
    
    return best_RT, best_pts3d

In [19]:
RT2, pts_cloud = double_disambiguation(K, RT1, K, RT2s, pts1, pts2, pts3d)
pts_cloud = pts_cloud[:, pts_cloud[2, :] > 0]

In [20]:
# Visualize 3D points
import plotly.graph_objects as go

# Assuming points_3D is your array of 3D points
x = pts_cloud[0]
y = pts_cloud[1]
z = pts_cloud[2]

fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z,
                                   mode='markers',
                                   marker=dict(size=2, color=z, colorscale='Viridis'))])

fig.update_layout(scene=dict(xaxis_title='X',
                             yaxis_title='Y',
                             zaxis_title='Z'))

fig.show()

In [21]:
# Initialize general Projection matrix list, RTs list and 3d points list
P_list = np.ndarray((images.shape[0], 3, 4))
P_done = np.full((images.shape[0], 1), False)
P_list[0], P_done[0] = K @ RT1, True
P_list[1], P_done[1] = K @ RT2, True

## Part 2: Increment SfM

### Find matches between image 3 and images 1 and 2

First we'll start one by one and then we'll do a more general form of this for N images.

In [ ]:
# Extract features
img3 = images[2]
kp, des = feature_extraction_set([img1, img2, img3])

# Match features
matches = feature_matching_set(kp, des)

common_matches1 = [m1 for m1 in matches[(0, 2)] for m2 in matches[(1, 2)] if m1.trainIdx == m2.trainIdx]
common_matches2 = [m2 for m1 in matches[(0, 2)] for m2 in matches[(1, 2)] if m1.trainIdx == m2.trainIdx]

### Projection Matrix

Next, we triangulate the position of the common points to get the 3D point corresponding to each 2D point we can. That way, we can use the DLT to get the projection matrix of the third image.

In [ ]:
pts1 = np.transpose([kp[0][m.queryIdx].pt for m in common_matches1])
pts2 = np.transpose([kp[1][m.queryIdx].pt for m in common_matches2])
pts3 = np.transpose([kp[2][m.trainIdx].pt for m in common_matches1])

pts3d = linear_triangulation2(P_list[-2], P_list[-1], pts1, pts2)

In [11]:
def calculate_projection_matrix(K, pts3d, pts2d):
    _, rod, T, _ = cv2.solvePnPRansac(pts3d.T, pts2d.T, K, None)#, flags=cv2.SOLVEPNP_P3P)
    R = cv2.Rodrigues(rod)[0]
    if np.linalg.det(R) < 0:
        R = R * -1
    P = K @ np.hstack((R, T))
    return P

In [ ]:
P_list.append(calculate_projection_matrix(pts3d, pts3))

### Triangulation

Finally, we triangulate the position of each matching points to get more 3D points we'll add to the pts_cloud and then we visualize the result.

In [ ]:
pts1 = np.transpose([kp[0][m.queryIdx].pt for m in matches[(0,2)]])
pts2 = np.transpose([kp[2][m.trainIdx].pt for m in matches[(0,2)]])

pts3d = linear_triangulation2(P_list[0], P_list[2], pts1, pts2)
pts_cloud = np.hstack((pts_cloud, pts3d))

pts1 = np.transpose([kp[1][m.queryIdx].pt for m in matches[(1,2)]])
pts2 = np.transpose([kp[2][m.trainIdx].pt for m in matches[(1,2)]])

pts3d = linear_triangulation2(P_list[1], P_list[2], pts1, pts2)
pts3d = pts3d[:, pts3d[2, :] > 0]
pts_cloud = np.hstack((pts_cloud, pts3d))

In [ ]:
# Visualize 3D points
import plotly.graph_objects as go

# Assuming points_3D is your array of 3D points
x = pts_cloud[0]
y = pts_cloud[1]
z = pts_cloud[2]

fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z,
                                   mode='markers',
                                   marker=dict(size=2, color=z, colorscale='Viridis'))])

fig.update_layout(scene=dict(xaxis_title='X',
                             yaxis_title='Y',
                             zaxis_title='Z'))

fig.show()

## Part 3: Generalization

So far, with only one sample we can see what the pattern would look like. So we start the main loop to get all the 3D points for the final model.

In [33]:
def plot_model(pts_cloud):
    # Assuming points_3D is your array of 3D points
    x = pts_cloud[0]
    y = pts_cloud[1]
    z = pts_cloud[2]

    fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z,
                                    mode='markers',
                                    marker=dict(size=2, color=z, colorscale='Viridis'))])

    fig.update_layout(scene=dict(xaxis_title='X',
                                yaxis_title='Y',
                                zaxis_title='Z'))

    fig.show()

In [57]:
def two_images_sfm(im1, im2, K):
    # Feature extraction
    kp, des = feature_extraction_set([im1, im2])

    # Feature matching
    matches = feature_matching_set(kp, des)

    # Fundamental matrix
    pts1 = np.transpose([kp[0][m.queryIdx].pt for m in matches[(0,1)]])
    pts2 = np.transpose([kp[1][m.trainIdx].pt for m in matches[(0,1)]])

    F = eight_point_algorithm(pts1, pts2)

    # Essential matrix
    E = essential_from_fundamental(K, F, K) # In this case, the same intrinsic values apply to all images

    # Get camera extrinsics from Essential matrix
    RT2s = pose_from_essential(E)

    # Define RT for camera 1 (center at world origin and matching orientation)
    RT1 = np.hstack((np.eye(3), np.zeros((3, 1))))
    
    RT2 = RT2s[0]

    pts3d = np.array([linear_triangulation(K, RT1, K, RT2, pts1, pts2) for RT2 in RT2s])

    RT2, pts_cloud = double_disambiguation(K, RT1, K, RT2s, pts1, pts2, pts3d)

    plot_model(pts_cloud)

    # Initialize general Projection matrix list, RTs list and 3d points list

    return K @ RT1, K @ RT2, pts_cloud

In [47]:
# Load all images
images = np.array(load_images_from_folder('dinos'))

In [48]:
# K = np.loadtxt('intrinsic_matrix.txt', dtype=float)

height, width = images.shape[1:3]
K = np.array([  # for dino
    [2360, 0, width / 2],
    [0, 2360, height / 2],
    [0, 0, 1]])

In [49]:
kp, des = feature_extraction_set(images)
matches = feature_matching_set(kp, des)

In [103]:
image_count = 0
pts_cloud = []
P_list = []

while image_count < images.shape[0]:
    P1, P2, pts3d = two_images_sfm(images[image_count], images[image_count+1], K)
    pts_cloud.append(pts3d)
    image_count += 2
    P_list.append([P1, P2])

    for i in range(image_count, images.shape[0]):
        idx1, idx2, idx3 = i-2, i-1, i
        image_count += 1
        
        # Get Common matches of 3 images
        common_matches1 = [m1 for m1 in matches[(idx1, idx3)] for m2 in matches[(idx2, idx3)] if m1.trainIdx == m2.trainIdx]
        common_matches2 = [m2 for m1 in matches[(idx1, idx3)] for m2 in matches[(idx2, idx3)] if m1.trainIdx == m2.trainIdx]
        
        if len(common_matches1) < 4:
            print('STOP')
            break

        # Triangulate common points
        common_pts1 = np.transpose([kp[idx1][m.queryIdx].pt for m in common_matches1])
        common_pts2 = np.transpose([kp[idx2][m.queryIdx].pt for m in common_matches2])
        common_pts3 = np.transpose([kp[idx3][m.trainIdx].pt for m in common_matches1])

        common_pts3d = linear_triangulation2(P_list[-1][-2], P_list[-1][-1], common_pts1, common_pts2)

        # Get Projection matrix
        P = calculate_projection_matrix(K, common_pts3d, common_pts3)

        P_list[-1].append(P)
        
        # Triangulate
        pts1_list = [np.transpose([kp[j][m.queryIdx].pt for m in matches[(j,idx3)]]) for j in range(idx1, idx3)]
        pts2_list = [np.transpose([kp[idx3][m.trainIdx].pt for m in matches[(j,idx3)]]) for j in range(idx1, idx3)]

        for j in range(2):
            pts3d = linear_triangulation2(P_list[-1][-3+j], P_list[-1][-1], pts1_list[j], pts2_list[j])
            pts_cloud[-1] = np.hstack((pts_cloud[-1], pts3d))

STOP


In [104]:
transf_list = []

for i in range(len(P_list)):
    RT = np.linalg.inv(K) @ P_list[i][-1]
    C = -(RT[:3, :3].T) @ RT[:3, -1]
    transf_list.append(np.vstack((np.hstack((RT[:3, :3], C[:, np.newaxis])), np.array([0, 0, 0, 1]))))

In [ ]:
# stop = False
# for i in range(2, images.shape[0]):
#     # Extract features
#     window = min(2, i)

#     # Get Common matches of 3 images
#     batrakin = 0
#     common_matches1 = [m1 for m1 in matches[(i-2-batrakin, i)] for m2 in matches[(i-1-batrakin, i)] if m1.trainIdx == m2.trainIdx]
#     common_matches2 = [m2 for m1 in matches[(i-2-batrakin, i)] for m2 in matches[(i-1-batrakin, i)] if m1.trainIdx == m2.trainIdx]
    
#     if len(common_matches1) < 4:
#         print('STOP')
#         break

#     # Triangulate common points
#     common_pts1 = np.transpose([kp[i-2-batrakin][m.queryIdx].pt for m in common_matches1])
#     common_pts2 = np.transpose([kp[i-1-batrakin][m.queryIdx].pt for m in common_matches2])
#     common_pts3 = np.transpose([kp[i][m.trainIdx].pt for m in common_matches1])

#     common_pts3d = linear_triangulation2(P_list[i-2-batrakin], P_list[i-1-batrakin], common_pts1, common_pts2)

#     # Get Projection matrix
#     P = calculate_projection_matrix(K, common_pts3d, common_pts3)

#     P_list[i], P_done[i] = P, True
    
#     # Triangulate
#     pts1_list = [np.transpose([kp[j][m.queryIdx].pt for m in matches[(j,i)]]) for j in range(i-window, i)]
#     pts2_list = [np.transpose([kp[i][m.trainIdx].pt for m in matches[(j,i)]]) for j in range(i-window, i)]

#     for j in range(window):
#         pts3d = linear_triangulation2(P_list[i-window+j], P_list[i], pts1_list[j], pts2_list[j])
#         # model_condition = ((pts3d[0, :] >= intervals_3d[0,0]) & (pts3d[0,:] <= intervals_3d[0,1]) &
#         #                    (pts3d[1, :] >= intervals_3d[1,0]) & (pts3d[1,:] <= intervals_3d[1,1]) &
#         #                    (pts3d[2, :] >= intervals_3d[2,0]) & (pts3d[2,:] <= intervals_3d[2,1]))
#         # pts3d = pts3d[:, model_condition]
#         pts_cloud = np.hstack((pts_cloud, pts3d))

In [105]:
for pts in pts_cloud:
    plot_model(pts)

In [106]:
for i in range(1, len(pts_cloud)):
    pts_cloud[i] = np.dot(transf_list[i-1], np.vstack((pts_cloud[i], np.ones(pts_cloud[i].shape[1]))))
    pts_cloud[i] = (pts_cloud[i] / pts_cloud[i][-1])[:3]

pts_cloud = np.hstack(pts_cloud)

In [107]:
plot_model(pts_cloud)